# **NO UTILIZADO**
# Demasiadas columnas finales


# Script para extraer datos utiles de cada tabla desde el formato zip directamente

Prueba de concepto para importar todos los files de la carpeta "tablas_paises", extraer en un loop todas las features y unirlas en un solo dataframe.

Luego se deberia hacer un proceso se limpieza para seleccionar las utiles. 

Tambien modificar los nombres de las variables para hacerlas mas "friendly"

In [77]:
import pandas as pd
import numpy as np
import zipfile
import io
import os

#supress warnings
import warnings
warnings.filterwarnings('ignore')

#save and load as parquet (for big files)
import pyarrow as pa
import pyarrow.parquet as pq

# Loop from ZIP file

In [78]:
# Initialize the DataFrame
all_data = pd.DataFrame()

# Define the directory containing the zip files
directory = r'..\data\FAOSTAT\all_raw\tablas_otras'

# Loop through all zip files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.zip'):
        zip_path = os.path.join(directory, filename)

        # Open the zip file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # List all files in the zip archive
            all_files = zip_ref.namelist()
            
            # Iterate through the file names in the zip archive
            for file_name in all_files:
                # Check if the file name contains the pattern "All_Data_" and ends with .csv
                if "All_Data_" in file_name and file_name.endswith(".csv"):
                    print(f"Found matching file: {file_name}")
                    # Read the CSV file into a DataFrame
                    with zip_ref.open(file_name) as file:
                        df = pd.read_csv(file, encoding="ISO-8859-1")

                        # Check conditions
                        condition_paises = all(col in df.columns for col in ['Area Code', 'Year'])
                        condition_comida = df['Item'].isin(['Meat of chickens; fresh or chilled', 'Tapioca of cassava', 'Wheat', 'Barley', 'Cattle']).any()

                        if condition_paises and not condition_comida:
                            print(f"La tabla {file_name} parece ser SOLO de PAISES")
                            print('\n', df.head())

                            
                            # Replace spaces and commas in column names
                            cols_with_spaces = ['Item', 'Element', 'Unit']
                            for col in cols_with_spaces:
                                df[col] = df[col].str.replace(' ', '_').str.replace(',', '_')

                            df['Element'] = df['Element'] + "_(" + df['Unit'] + ")"

                            # Extract all unique items and elements
                            unique_items = df['Item'].unique()
                            unique_elements = df['Element'].unique()

                            # Filter the DataFrame
                            filtered_df = df[(df['Element'].isin(unique_elements)) & (df['Item'].isin(unique_items))]

                            # Group by Area, Year, Item, and Element, then sum the values
                            grouped = filtered_df.groupby(['Area Code', 'Year', 'Item', 'Element'])['Value'].sum().reset_index()

                            # Pivot the table to have combinations of Items and Elements as columns
                            pivot_df = grouped.pivot_table(index=['Area Code', 'Year'], 
                                                           columns=['Item', 'Element'], 
                                                           values='Value', 
                                                           aggfunc='sum').reset_index()

                            # Flatten the multi-level column index and rename columns
                            pivot_df.columns = ['_'.join(col).strip() if col[1] else col[0] for col in pivot_df.columns.values]

                            # Merge the pivot_df with all_data on 'Area Code' and 'Year'
                            if all_data.empty:
                                all_data = pivot_df
                            else:
                                all_data = pd.merge(all_data, pivot_df, on=['Area Code', 'Year'], how='outer')

# Reset the index of the final DataFrame
all_data.reset_index(drop=True, inplace=True)

Found matching file: Food_Security_Data_E_All_Data_(Normalized).csv
La tabla Food_Security_Data_E_All_Data_(Normalized).csv parece ser SOLO de PAISES

    Area Code Area Code (M49)         Area Item Code  \
0          2            '004  Afghanistan     21010   
1          2            '004  Afghanistan     21010   
2          2            '004  Afghanistan     21010   
3          2            '004  Afghanistan     21010   
4          2            '004  Afghanistan     21010   

                                                Item  Element Code Element  \
0  Average dietary energy supply adequacy (percen...          6121   Value   
1  Average dietary energy supply adequacy (percen...          6121   Value   
2  Average dietary energy supply adequacy (percen...          6121   Value   
3  Average dietary energy supply adequacy (percen...          6121   Value   
4  Average dietary energy supply adequacy (percen...          6121   Value   

   Year Code       Year Unit Value Flag Note  
0

In [82]:
all_data.shape

(10956, 129)

In [79]:
all_data.head()

,Area Code,Year,Average_dietary_energy_requirement_(kcal/cap/day)_Value_(kcal/cap/d),Average_dietary_energy_supply_adequacy_(percent)_(3-year_average)_Value_(%),Average_fat_supply_(g/cap/day)_(3-year_average)_Value_(g/cap/d),Average_protein_supply_(g/cap/day)_(3-year_average)_Value_(g/cap/d),Average_supply_of_protein_of_animal_origin_(g/cap/day)_(3-year_average)_Value_(g/cap/d),Cereal_import_dependency_ratio_(percent)_(3-year_average)_Value_(%),Coefficient_of_variation_of_habitual_caloric_consumption_distribution_(real_number)_Value_(No),Gross_domestic_product_per_capita__PPP__(constant_2017_international_$)_Value_(Int$/cap),...,Prevalence_of_severe_food_insecurity_in_the_town_and_semi-dense_area_adult_population_(percent)_(annual_value)_Confidence_interval:_Upper_bound_(%),Prevalence_of_severe_food_insecurity_in_the_town_and_semi-dense_area_adult_population_(percent)_(annual_value)_Value_(%),Prevalence_of_severe_food_insecurity_in_the_urban_adult_population_(percent)_(annual_value)_Confidence_interval:_Lower_bound_(%),Prevalence_of_severe_food_insecurity_in_the_urban_adult_population_(percent)_(annual_value)_Confidence_interval:_Upper_bound_(%),Prevalence_of_severe_food_insecurity_in_the_urban_adult_population_(percent)_(annual_value)_Value_(%),Prevalence_of_undernourishment_(percent)_(3-year_average)_Value_(%),Prevalence_of_undernourishment_(percent)_(annual_value)_Value_(%),Rail_lines_density_(total_route_in_km_per_100_square_km_of_land_area)_Value_(km),Share_of_dietary_energy_supply_derived_from_cereals__roots_and_tubers_(kcal/cap/day)_(3-year_average)_Value_(%),Value_of_food_imports_in_total_merchandise_exports_(percent)_(3-year_average)_Value_(%)
0,1,2000,2369,NaN,NaN,NaN,NaN,NaN,0.22,3921.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.8,NaN,NaN
1,1,2000-2002,NaN,95,44.3,65.3,20.7,61.7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,25.7,NaN,NaN,60,42
2,1,2001,2385,NaN,NaN,NaN,NaN,NaN,0.23,4346.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.8,NaN,NaN
3,1,2001-2003,NaN,97,49.6,67,22.7,53.9,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,24.2,NaN,NaN,57,31
4,1,2002,2401,NaN,NaN,NaN,NaN,NaN,0.24,4965.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.4,NaN,NaN


# Save to parquet

In [80]:
#all_data.to_parquet(r'..\data\FAOSTAT\clean_data\all_tables_processed\tablas_paises\all_country_features.parquet', index=False)

# Loop from CSVs

In [81]:

"""# Initialize the DataFrame
all_data = pd.DataFrame()

# Define the directory containing the CSV files
directory = r'..\data\FAOSTAT\sample_data\tablas_paises'

# Loop through all CSV files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        path = os.path.join(directory, filename)
        df = pd.read_csv(path)

        # Check conditions
        condition_paises = all(col in df.columns for col in ['Area Code', 'Year'])
        condition_comida = df['Item'].isin(['Meat of chickens; fresh or chilled', 'Tapioca of cassava', 'Wheat', 'Barley', 'Cattle']).any()

        if condition_paises and not condition_comida:
            print(f"La tabla {filename} parece ser SOLO de PAISES")
            print('\n', df.head())
            
            # Replace spaces and commas in column names
            cols_with_spaces = ['Item', 'Element', 'Unit']
            for col in cols_with_spaces:
                df[col] = df[col].str.replace(' ', '_').str.replace(',', '_')

            df['Element'] = df['Element'] + "_(" + df['Unit'] + ")"

            # Extract all unique items and elements
            unique_items = df['Item'].unique()
            unique_elements = df['Element'].unique()

            # Filter the DataFrame
            filtered_df = df[(df['Element'].isin(unique_elements)) & (df['Item'].isin(unique_items))]

            # Group by Area, Year, Item, and Element, then sum the values
            grouped = filtered_df.groupby(['Area Code', 'Year', 'Item', 'Element'])['Value'].sum().reset_index()

            # Pivot the table to have combinations of Items and Elements as columns
            pivot_df = grouped.pivot_table(index=['Area Code', 'Year'], 
                                           columns=['Item', 'Element'], 
                                           values='Value', 
                                           aggfunc='sum').reset_index()

            # Flatten the multi-level column index and rename columns
            pivot_df.columns = ['_'.join(col).strip() if col[1] else col[0] for col in pivot_df.columns.values]

            # Append the data to all_data DataFrame
            if all_data.empty:
                all_data = pivot_df
            else:
                all_data = pd.merge(all_data, pivot_df, on=['Area Code', 'Year'], how='outer')

# Reset the index of the final DataFrame
all_data.reset_index(drop=True, inplace=True)"""

'# Initialize the DataFrame\nall_data = pd.DataFrame()\n\n# Define the directory containing the CSV files\ndirectory = r\'..\\data\\FAOSTAT\\sample_data\tablas_paises\'\n\n# Loop through all CSV files in the directory\nfor filename in os.listdir(directory):\n    if filename.endswith(\'.csv\'):\n        path = os.path.join(directory, filename)\n        df = pd.read_csv(path)\n\n        # Check conditions\n        condition_paises = all(col in df.columns for col in [\'Area Code\', \'Year\'])\n        condition_comida = df[\'Item\'].isin([\'Meat of chickens; fresh or chilled\', \'Tapioca of cassava\', \'Wheat\', \'Barley\', \'Cattle\']).any()\n\n        if condition_paises and not condition_comida:\n            print(f"La tabla {filename} parece ser SOLO de PAISES")\n            print(\'\n\', df.head())\n            \n            # Replace spaces and commas in column names\n            cols_with_spaces = [\'Item\', \'Element\', \'Unit\']\n            for col in cols_with_spaces:\n        